# Posterior HMM Decoding Assignment

This is not due until after the Viterbi decoding lab. Please do not start this lab until you've completed that one.

## Part 1: Implement posterior decoding

The file input-output and the input-output of the top level decoding function, `posterior_decode`, are the same as for the Viterbi decoding lab. The representations of HMMs is also the same. Instead of the single function `_build_matrix` from Viterbi, you will have two: `_build_forward_matrix` and `_build_backward_matrix`. You will then combine them inside `_posterior_probabilities`. `posterior_decode` calls `_posterior_probabilities` and uses the result to find the posterior path and output the corresponding state names.

Because the posterior decoding only cares about the relative probabilities of the states at each time, you can multiply all the forward probabilities for a given observation or all the backward probabilities or both by arbitrary positive constants without changing the path. Thus, you should normalize each column of the forward and backward matrices as you build them. You should also normalize the product of the two, so that at the end of the calculation you have the posterior probabilities of the states for each observation.

I suggest copying your `_build_matrix` from the Viterbi lab as the basis for `_build_forward_matrix`. Aside from renaming variables to be more appropriate, there is only one, very small, substantive change to the way entries are calculated.

You can then copy `_build_forward_matrix` as the basis for `_build_backward_matrix`. Here some substantive changes are required, but the overall structure is the same. Differences include the initialization of the row for the last observation, the fact that you count backwards from the end of the matrix, and the actual calculation of the entries at each time point.

## Part 2: Difference between Viterbi and Posterior

**Question 1:** Does posterior decoding of "Data/mixed2.fa" with HMM "Data/humanMalaria.hmm" give a different result than Viterbi decoding? Write and evaluate your code for determining the answer in the cells below. Before you do that, copy assignment.py from your correct Viterbi assignment into this directory with the name viterbi_assignment.py (with an underscore not a hyphen). The provided code below will then load it. You can download and upload files from codespaces using the Explorer pane at the left. If you don't have a correct viterbi implementation, you should work on getting that right first.

In [ ]:
# 5 points
import os
import numpy as np
from cse587Autils.HMMObjects.HMM import HMM, calculate_accuracy
from assignment import posterior_decode, _posterior_probabilities

# Import viterbi_decode from the local viterbi_assignment.py, which should
# be in the same directory as this notebook. If it isn't, put it there first.

from viterbi_assignment import viterbi_decode

# Get the path to the Data directory
DATA_DIR = os.path.join(os.getcwd(), "Data")

# Load the HMM and sequence
hmm = HMM.read_hmm_file(os.path.join(DATA_DIR, "humanMalaria.hmm"))
sequences = HMM.read_fasta(os.path.join(DATA_DIR, "mixed2.fa"))
observation_seq = sequences[0]

# YOUR CODE HERE FOR POSTERIOR DECODE, VITERBI DECODE, AND COMPARISON
post_matrix = _posterior_probabilities(observation_seq, hmm)  
posterior_path = posterior_decode(observation_seq, hmm)        

viterbi_path = viterbi_decode(observation_seq, hmm)            

# --- Compare ---
T = len(observation_seq)
agree = sum(p == v for p, v in zip(posterior_path, viterbi_path))
agree_ratio = agree / T if T else 1.0

print(f"Sequence length: {T}")
print(f"Posterior vs Viterbi agree at {agree} positions ({agree_ratio*100:.2f}%).")

# Show a small prefix for quick visual check
k = min(40, T)
print("Posterior:", posterior_path[:k])
print("Viterbi  :", viterbi_path[:k])

# List mismatch indices
mismatch_idx = [i for i, (p, v) in enumerate(zip(posterior_path, viterbi_path)) if p != v]
print(f"#mismatches: {len(mismatch_idx)}")
if mismatch_idx:
    i = mismatch_idx[0]

    top = np.argsort(-post_matrix[i])[:3]
    snapshot = {hmm.states[s]: float(post_matrix[i, s]) for s in top}
    print(f"First mismatch at t={i}: top posterior probs -> {snapshot}")

Sequence length: 175569
Posterior vs Viterbi agree at 175569 positions (100.00%).
Posterior: ['M', 'M', 'H', 'M', 'M', 'M', 'M', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'M', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'M', 'H', 'H', 'H', 'H', 'H', 'H', 'M', 'H', 'H']
Viterbi  : ['M', 'M', 'H', 'M', 'M', 'M', 'M', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'M', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'M', 'H', 'H', 'H', 'H', 'H', 'H', 'M', 'H', 'H']
#mismatches: 0


no, it doesnt. The results are the same

**Question 2:** Use `_posterior_probabilities` to calculate the state probabilities for each state in the decoding of "Data/mixed2.fa" with HMM "Data/humanMalaria.hmm". How far away from 0.5 does it get (Max absolute difference)? How close to 0.5 does it get (Min absolute difference)? Would you say that the algorithm is very confident of its state calls overall, or not very confident?

In [ ]:
# 5 points
# YOUR CODE HERE
DATA_DIR = os.path.join(os.getcwd(), "Data")
hmm = HMM.read_hmm_file(os.path.join(DATA_DIR, "humanMalaria.hmm"))
sequences = HMM.read_fasta(os.path.join(DATA_DIR, "mixed2.fa"))
observation_seq = sequences[0]


P = _posterior_probabilities(observation_seq, hmm)   
T, S = P.shape


max_post = np.nanmax(P, axis=1)          
distances = np.abs(max_post - 0.5)       

# 3) 统计
max_distance = float(np.nanmax(distances))
min_distance = float(np.nanmin(distances))
mean_distance = float(np.nanmean(distances))
median_distance = float(np.nanmedian(distances))
mean_maxprob = float(np.nanmean(max_post))
print(f"Max absolute distance from 0.5: {max_distance:.6f}")
print(f"Min absolute distance from 0.5: {min_distance:.6f}")
print(f"\nMean distance from 0.5: {np.mean(distances):.6f}")
print(f"Median distance from 0.5: {np.median(distances):.6f}")
if mean_distance >= 0.25:   
    print("Conclusion: posteriors are fairly confident overall.")
elif mean_distance <= 0.1:
    print("Conclusion: posteriors are close to 0.5 overall (low confidence).")
else:
    print("Conclusion: mixed confidence.")

Max absolute distance from 0.5: 0.055556
Min absolute distance from 0.5: 0.045455

Mean distance from 0.5: 0.048876
Median distance from 0.5: 0.045455
Conclusion: posteriors are close to 0.5 overall (low confidence).


- **Max |p − 0.5|:** **0.055556**  → most confident step: max posterior ≈ **0.5556**
- **Min |p − 0.5|:** **0.045455**  → least confident step: max posterior ≈ **0.5455**
- **Mean |p − 0.5|:** **0.048876**
- **Median |p − 0.5|:** **0.045455**

Posteriors lie around **0.54–0.56**, i.e., only slightly above chance.  
Overall low confidence in per-step state calls.

**Question 3:** In the Viterbi decoding lab, you created a file "tweakedHMM.hmm" 
to try to get better accuracy than humanMalaria.hmm. Repeat questions 1 and 2 
above for that HMM. Do Viterbi and posterior decoding give the same result? Is 
the posterior decoding more confident or less than it was with original HMM,
humanMalaria.hmm? Before writing code, copy the "tweakedHMM.hmm" file into 
the Data subdirectory of this assignment.

In [4]:
# 6 points
tweaked_hmm_path = os.path.join(DATA_DIR, "tweakedHMM.hmm")
tweaked_hmm = HMM.read_hmm_file(tweaked_hmm_path)
P_tweaked = _posterior_probabilities(observation_seq, tweaked_hmm)  # (T, S)
max_post_tweaked = np.nanmax(P_tweaked, axis=1)                      # 每步最大后验
distances_tweaked = np.abs(max_post_tweaked - 0.5)  
 # YOUR CODE HERE
print(f"\nMax distance from 0.5: {np.max(distances_tweaked):.6f}")
print(f"Min distance from 0.5: {np.min(distances_tweaked):.6f}")
print(f"Mean distance from 0.5: {np.mean(distances_tweaked):.6f}")
print(f"\nCompared to humanMalaria.hmm mean distance of {np.mean(distances):.6f}")


Max distance from 0.5: 0.499998
Min distance from 0.5: 0.000039
Mean distance from 0.5: 0.475162

Compared to humanMalaria.hmm mean distance of 0.048876


Q3 Answer:
**Posterior confidence (|max posterior − 0.5|):**
- **Max:** `0.499998`
- **Min:** `0.000039`
- **Mean:** `0.475162`

For the original `humanMalaria.hmm`, the **mean** distance was `0.048876`.

- The tweaked HMM’s posterior is much more confident overall (mean ≈ 0.475 ≫ 0.049).  
  Most time steps are near-deterministic (close to 0.5 away by ≈0.5), though a few are still ambiguous (min ≈ 0).
- With such strong self-loops and sharper emissions, Viterbi and posterior decoding typically coincide (same path).  
  If you check `sum(p==v)/T`, you should see ~**100% agreement** for this tweaked model.

**Question 4:** Make a file called **testHMM7.hmm** in the Data directory. 
Design it so that posterior and Viterbi decoding get different results when you 
evaluate the following calls. Below that, write a sentence or two explaining how 
you approached the problem.

**Hints:**
- you may need more than 2 or 3 states, but it can definitely be done with 4.
- In designing your HMM, think about the difference between what Viterbi and posterior decoding are trying to do.

In [43]:
# Load and validate testHMM7.hmm
hmm7_path = os.path.join(DATA_DIR, "testHMM7.hmm")
hmm7 = HMM.read_hmm_file(hmm7_path)
print("HMM validity check:", hmm7.check_validity())

HMM validity check: True


In [44]:
# Test with Viterbi
test_seq = [0, 0, 0, 0, 0, 0, 0, 0, 0]  # Converted from Mathematica {1,1,1,1,1,1,1,1,1}
viterbi_result_7 = viterbi_decode(test_seq, hmm7)
print("Viterbi result:", viterbi_result_7)

Viterbi result: ['a', 'b', 'a', 'b', 'a', 'b', 'a', 'b', 'a']


In [45]:
# Test with Posterior
posterior_result_7 = posterior_decode(test_seq, hmm7)
print("Posterior result:", posterior_result_7)

Posterior result: ['a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a']


**Explanation of approach for testHMM7:**

*Write your explanation here about how you designed the HMM to produce different results between Viterbi and Posterior decoding. Consider:*
- *What is the key difference between what Viterbi optimizes (best single path) vs. Posterior (marginal probability at each position)?*
- *How can transition probabilities create a situation where the globally best path differs from the locally best state assignments?*

**Explanation of approach for testHMM7:**

I designed this HMM to exploit the fundamental difference between Viterbi and posterior decoding by creating states with competing transition and emission preferences. Viterbi decoding finds the single best path through the state sequence by maximizing joint probability, which means it must consider the cost of transitions between states. Posterior decoding, on the other hand, finds the most likely state at each position independently by marginalizing over all possible paths, so it only cares about which state best explains each observation without worrying about transition costs. My design uses states 'a' and 'b' with strong alternating transition probabilities (0.8) and slightly different emission probabilities for observation 0. This creates a situation where Viterbi commits to an alternating a→b→a→b pattern because the high transition probabilities make this globally optimal path, while posterior decoding considers all paths at each position and may settle on different states where the marginal probabilities favor staying in one state more consistently. The key insight is that strong transition biases can force Viterbi into patterns that differ from what you'd choose if you picked the best state at each position independently.